# <font color="#76b900">**Notebook 0:** First Contact With NIMs</font>

**Welcome to the introductory notebook on NVIDIA Inference Microservices (NIMs).** In this notebook, we will explore how to interact with a NIM endpoint, specifically focusing on the Llama 3 8B model. By the end of this notebook, you will have a foundational understanding of NIMs and how to perform basic tasks such as checking the status of a NIM endpoint, querying available models, and generating text.

## Learning Objectives

By the end of this notebook, you will be able to:

- Interact with a NIM endpoint to check its status and the available models.
- Call a NIM endpoint of Llama 3 8B to generate text using curl and Python.
- Understand the difference between end-to-end latency and time to first token.

**Before starting this notebook, please make sure to watch its corresponding video.**

## Table of Contents
- [**Getting Started With Your First NIM**](#Getting-Started-With-Your-First-NIM)
- [**End-To-End Latency versus Time-to-First-Token**](#End-To-End-Latency-versus-Time-to-First-Token)
- [**[EXERCISE] Experimental Setup**](#[EXERCISE]-Experimental-Setup)
- [**Next Steps**](#Next-Steps)

<br><hr>

## **Getting Started With Your First NIM**

[**NVIDIA NIMs**](https://www.nvidia.com/en-us/ai/) are microservices that kickstart GPU-optimized processes and conform to your system requirements for optimized delivery. They can be tested at [**build.nvidia.com**](https://www.build.nvidia.com) and span single-LLM (Llama, Mixtral, Nemotron, ...), non-linguistic models (diffusion, vision, health, ...), and orchestration (retriever) configurations. Additionally, they can be accessed via [**NGC**](https://catalog.ngc.nvidia.com) and kickstarted with relative ease. 

- **To get started with this course:** A NIM has already been kickstarted for you and will be up soon! Feel free to check out [**99-Reading-Logs.ipynb**](./99-Reading-Logs.ipynb) to check on its status. To see more about how this particular instance was deployed, check out [**`composer/docker-compose.yml`**](composer/docker-compose.yml).
- **To learn about how to set one up with regular docker access:** Please visit [**99-Deploy-Llama-NIM.ipynb**](./99-Deploy-Llama-NIM.ipynb) to see how you could take advantage of NIMs using base Docker. 

Once the service is live, we can check that NIM is available as a `nim` service running in the background from another container. From the host machine, you should change the URL from `nim` to `localhost`.

In [1]:
!curl http://nim:8000/v1/health/ready

{"object":"health.response","message":"Service is ready."}

Assuming that the microservice is ready, we should get a response similar to `{"object": "health-response", "message": "Service is ready."}`. If you do not see this, check back on your deployment in [**99-Deploy-Llama-NIM.ipynb**](./99-Deploy-Llama-NIM.ipynb) and see if something went wrong.

Next, let's verify the model loaded into NIM:

In [2]:
!curl -s -X GET 'http://nim:8000/v1/models' | jq -r '.data[0].id'

meta/llama3-8b-instruct


You should see that `meta/llama3-8b-instruct` is available, and we will call it to generate responses. Let's test it by performing a simple inference request via its most basic interface; a direct curl request:

In [3]:
%%time
%%bash
# OPTION 1: Send requests in terminal via `curl`
curl -s -X 'POST' \
    'http://nim:8000/v1/completions' \
    -H 'accept: application/json' \
    -H 'Content-Type: application/json' \
    -d '{
"model": "meta/llama3-8b-instruct",
"prompt": "Could you explain what a GPU is?",
"max_tokens": 30
}' | jq -r '.choices[0].text'

 A Graphics Processing Unit (GPU) is a specialized electronic circuit designed to quickly manipulate and alter memory to accelerate the creation of images in a frame buffer intended
CPU times: user 1.47 ms, sys: 4.15 ms, total: 5.63 ms
Wall time: 436 ms


<br>

Feel free to modify the prompt of the number of max_tokens to produce different text. When increasing the number of max_tokens, you will notice that the response takes longer. For example, let's increase the number of max_tokens to 300 and time the request:

In [4]:
%%time
## OPTION 1: continue with `curl`
# %%bash
# TIMEFORMAT="%Es" curl -s -X 'POST' \ -s -X 'POST' \
#     'http://nim:8000/v1/completions' \
#     -H 'accept: application/json' \
#     -H 'Content-Type: application/json' \
#     -d '{
# "model": "meta/llama3-8b-instruct",
# "prompt": "Could you explain what a GPU is?",
# "max_tokens": 300
# }' | jq -r '.choices[0].text'

## Option 2: Continue in python via `requests`
import requests
import json

url = "http://nim:8000/v1/completions"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
data = {
    "model": "meta/llama3-8b-instruct",
    "prompt": "Could you explain what a GPU is?",
    "max_tokens": 300,
}

response = requests.post(url, headers=headers, data=json.dumps(data))

if response.status_code == 200:
    response_text = response.json().get('choices', [{}])[0].get('text', '').strip()
    print(f"Response:\n{response_text}")
else:
    print(f"Failed to get a response, status code: {response.status_code}")

Response:
A GPU is abbreviated as Graphics Processing Unit. It is a crucial component of a computer system, responsible for handling and rendering graphical and computationally intensive tasks. In this post, you will learn about what a GPU is, its role in computers, and why it is essential for many modern applications.

What is a GPU?
----------------

A GPU (Graphics Processing Unit) is a specialized electronic circuit designed to quickly manipulate and alter memory to accelerate the creation of images in a frame buffer intended for output to a display device. Graphics Processing Units have evolved to become highly parallel processing units, with thousands of processing cores, capable of executing many instructions simultaneously.

Architecture of a GPU
-----------------

GPUs are typically comprised of multiple components, including:

1. **CUDA Cores or Stream Processors**: These are the primary processing units responsible for performing calculations. The number of CUDA cores or str

<br>

In our setup, the longer response takes around 3.2s, vs 0.6s for the shorter one. Note that the `max_tokens` number of tokens may not be reached if an end-of-sequence token is returned by the LLM before. 

<br><hr>

## **End-To-End Latency versus Time-to-First-Token**

An inference request has two main stages: **prefill** and **decoding**.
- During **prefill**, the LLM processes the prompt we send to the model, which in our example was "Could you explain what a GPU is?", and produces the first generated token.
    - `TTFT`: **time-to-first-token**, or prefill duration.
- During **decoding**, the LLM predicts the subsequent tokens one at a time, until reaching max_tokens or producing an end-of-sequence token. Prefill and decoding phases happen internally even if only the completed response is returned, as above.
    - `E2E Latency`: **End-to-end latency** from the combined prefill and decoding stages.

The latency you measured before was the `E2E Latency`, but it will be better for us to focus on `TTFT` for streaming applications like chatbots. Since the user can start reading the response as the LLM generates tokens, the user experience is acceptable as long as the speed of token generation is faster than the human reading speed. 

Below, we connect to our NIM microservice via the OpenAI client for simplicity, swapping the `base_url` to our local endpoint and setting `stream=True` to invoke streaming:

In [5]:
from openai import OpenAI
import time
import sys

client = OpenAI(base_url="http://nim:8000/v1", api_key="not-used")
response = client.chat.completions.create(
    model="meta/llama3-8b-instruct",
    messages=[
        {'role': 'user', 'content': 'Tell me a long story about GPUs'}
    ],
    max_tokens=300,
    stream=True
)
    
for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        sys.stdout.write(content)
        sys.stdout.flush()

It was the early 1990s, and the world of computing was on the cusp of a revolution. The once-dominant CPU (Central Processing Unit) was about to meet its match in a new kind of processor: the GPU (Graphics Processing Unit).

In those days, the graphics capabilities of computers were limited to simple, 2D graphics. But a small startup called NVIDIA, founded by Jensen Huang and Chris Malachowsky, had a vision to change all that. Their goal was to create a processor that could handle complex, 3D graphics with ease.

The first generation of NVIDIA's GPUs, the RIVA 128, was released in 1997. It was a game-changer. No longer would gamers and graphics enthusiasts have to settle for bland, 2D graphics. The RIVA 128 brought 3D rendering to the masses, and with it, a whole new world of possibilities opened up.

As the years went by, NVIDIA continued to push the boundaries of what was possible with GPUs. They introduced new architectures like the GeForce and the Quadro, each designed to handle mo

<br>

You can see that a user could enjoy a decent reading experience even as the model generates more and more of the response. This resembles the experience of interacting with an online LLM inference application like a chatbot, which explains why `TTFT` is so important there.

Notice that our prompt was a very short "Tell me a long story about GPUs" request. As the length prompt increases, we should expect the `TTFT` to go up since more and more context tokens have to be propagated through our network. Perhaps the prompt length should be a factor in our timing considerations...

<br>

## **[EXERCISE] Experimental Setup**

To reinforce this point, let's modify the previous code to streamline experimentation. Please define the `measure_latency` command below to help you perform some simple timing experiments based on a set of reasonable parameters.

In [6]:
import time

def measure_latency(n_input_tokens: int = 50, max_output_tokens: int = 1, verbose = True):
    # Let's define a dummy prompt with a variable number of input tokens
    dummy_prompt = " ".join(["hi"] * (n_input_tokens - 1))

    ## TODO: Create a connector that connects to the nim endpoint
    client = None
    
    # Record the start time and (later) the end time of the simulation to compute duration
    start_time = time.time()
    
    ## TODO: Using client, send the request to the NIM
    ## - Make sure to set max_tokens to stop generation
    ## - Make sure stream=True as the following code expects a generator
    ## - Set messages=[...] with the openai library
    ##    - But if you notice you're not getting enough outputs, use "prompt": "..."  with the requests library
    response = None
    
    ## Wait for the responses to come in, accumulating them along the way
    ## NOTE: If using Chat endpoint, the first token is generally a confirmation empty-token. Best solution is to check if token has content
    n_generated_tokens = 0
    # n_generated_tokens = -1
    for chunk in response:
        n_generated_tokens += 1
    duration = time.time() - start_time
    
    if verbose: 
        print(f"{n_generated_tokens}-token latency with {n_input_tokens} input tokens is {duration:.2f}s")
    
    return duration

The previous code measures the time to generate a response according to the specified `n_input_tokens` and `max_output_tokens`. We are not printing the response since we employ a dummy input prompt formed by repeated "hi"s. Note that the latency of the LLM doesn't depend on the type of token passed into it: as a result, it doesn't really matter what tokens are contained in the prompt. For latency purposes, the only important variable is the number of tokens in the prompt.
Let's time the function call with the default `max_output_tokens=1` to measure the `TTFT`:

In [7]:
## TODO: Measure the latency of 50-in, 1-out

As the number of input tokens increases, so does the `TTFT`. For example, let's set 8000 input tokens:

In [8]:
## TODO: Measure the latency of 8000-in, 1-out

In our setup, the `TTFT` increased from 0.03s to 0.6s, which isn't too terrible but still a sizable increase. Now, consider what happens when we try to generate 500 tokens:

In [9]:
## TODO: Measure the latency of 50-in, 500-out

**In our setup, assuming no extraneous load from other services/users:**
 - The `TTFT` for just 50 input tokens takes ~0.02s. There's very little prefill to process and very little decoding to do, so this makes a lot of sense. 
 - The `TTFT` for 8000 input tokens takes ~0.6s. This is quite a reasonable number for user experience, but the impact is sizable.
 - The `E2E Latency` of just 50 input tokens but 500 output tokens takes ~6.6s. This is because the prefill stage is more efficient than decoding. This will be discussed in detail in the next notebook but has to do with the one-at-a-time nature of autoregressive decoding.

That's why streaming is so powerful in improving user experience. We encourage you to try measuring the `TTFT` and `E2E Latency` for other use cases by changing `n_input_tokens` and `max_output_tokens` in the function `measure_latency()`.

<details>
<summary><b>Reveal Solution</b></summary>

```python 
import time

def measure_latency(n_input_tokens: int = 50, max_output_tokens: int = 1, verbose = True):
    # Let's define a dummy prompt with a variable number of input tokens
    dummy_prompt = " ".join(["hi"] * (n_input_tokens - 1))

    ## TODO: Create a connector that connects to the nim endpoint
    client = OpenAI(base_url="http://nim:8000/v1", api_key="not-used")
    
    # Record the start time and (later) the end time of the simulation to compute duration
    start_time = time.time()
    
    ## TODO: Using client, send the request to the NIM
    ## - Make sure to set max_tokens to stop generation
    ## - Make sure stream=True as the following code expects a generator
    ## - Set messages=[...] with the openai library
    ##    - But if you encounter errors, use "prompt": "..."  with the requests library
    # response = client.chat.completions.create(
    response = client.completions.create(
        model="meta/llama3-8b-instruct",
        # messages = [{'role': 'user', 'content': dummy_prompt}],
        prompt = dummy_prompt,
        max_tokens = max_output_tokens,
        stream = True,
    )
    ## NOTE: With messages, models have strong priors to stop generating text "in a conversational fashion"
    ## As such, trying to use messages here will result in shorter generations which will mess up the benchmarks.
    ## Later notebooks address this issue by explicitly disabling end-of-string tokens under the hood. 

    ## Wait for the responses to come in, accumulating them along the way
    ## NOTE: If using Chat endpoint, the first token is generally a confirmation empty-token. Best solution is to check if token has content
    n_generated_tokens = 0
    # n_generated_tokens = -1
    for chunk in response:
        n_generated_tokens += 1
    duration = time.time() - start_time
    
    if verbose: 
        print(f"{n_generated_tokens}-token latency with {n_input_tokens} input tokens is {duration:.2f}s")
    
    return duration

## TODO: Measure the latency of 50-in, 1-out
measure_latency(50);

## TODO: Measure the latency of 8000-in, 1-out
measure_latency(8000);

## TODO: Measure the latency of 50-in, 500-out
measure_latency(50, 500);
```

</details>